# Notebook to make Fig. 1

In [1]:
# Import necessary packages & set-up plotting aesthetics

import numpy as np 
import pylab
import pandas as pd
import lal
from scipy.stats import gaussian_kde
import imageio

import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
from matplotlib import style

style.use('plotting.mplstyle')

plt.rc('text', usetex=True)
matplotlin.rcParams['text.latex.preamble'] = "" ## TODO: get this to work with LaTeX

### Load in data: posteriors

Load in posterior and prior samples

In [2]:
pe_output_dir = '/home/simona.miller/gw190521-td/output/'
path_template = pe_output_dir+'{0}_gw190521_{1}_NRSur7dq4_dec8_flow11_fref11_{2}_TstartTend.dat'

date = '042823'
runs = ['insp', 'rd']
tcutoffs = ['m50M', 'm40M', 'm37.5M', 'm35M', 'm32.5M', 'm30M', 'm27.5M', 'm25M', 'm22.5M', 'm20M', 
            'm17.5M', 'm15M', 'm12.5M', 'm10M', 'm7.5M', 'm5M', 'm2.5M', '0M', '2.5M', '5M', '7.5M', 
            '10M', '12.5M', '15M', '17.5M', '20M', '30M', '40M', '50M']

# samples from the various time cuts
paths = {}
for run in runs: 
    for tcut in tcutoffs: 
        key = f'{run} {tcut}'
        paths[key] = path_template.format(date,run,tcut)

# samples from full duration (no time cut) 
paths['full'] = path_template.format(date,'full','0M')

# prior samples
paths['prior'] = pe_output_dir+'gw190521_sample_prior.dat'

td_samples = {}
for k, p in paths.items():
    try:
        td_samples[k] = np.genfromtxt(p, names=True, dtype=float)
    except:
        print(f'could not find {p}')

could not find /home/simona.miller/gw190521-td/output/042823_gw190521_insp_NRSur7dq4_dec8_flow11_fref11_m50M_TstartTend.dat
could not find /home/simona.miller/gw190521-td/output/042823_gw190521_insp_NRSur7dq4_dec8_flow11_fref11_m40M_TstartTend.dat
could not find /home/simona.miller/gw190521-td/output/042823_gw190521_insp_NRSur7dq4_dec8_flow11_fref11_m37.5M_TstartTend.dat
could not find /home/simona.miller/gw190521-td/output/042823_gw190521_insp_NRSur7dq4_dec8_flow11_fref11_m35M_TstartTend.dat
could not find /home/simona.miller/gw190521-td/output/042823_gw190521_insp_NRSur7dq4_dec8_flow11_fref11_m32.5M_TstartTend.dat
could not find /home/simona.miller/gw190521-td/output/042823_gw190521_insp_NRSur7dq4_dec8_flow11_fref11_m30M_TstartTend.dat
could not find /home/simona.miller/gw190521-td/output/042823_gw190521_insp_NRSur7dq4_dec8_flow11_fref11_m27.5M_TstartTend.dat
could not find /home/simona.miller/gw190521-td/output/042823_gw190521_insp_NRSur7dq4_dec8_flow11_fref11_m25M_TstartTend.dat
co

Define functions to calculate component masses and $\chi_\mathrm{p}$

In [3]:
# Function to component masses 
def m1m2_from_mtotq(mtot, q):
    m1 = mtot / (1 + q)
    m2 = mtot - m1
    return m1, m2

# Function to calculate chi_p 
def chi_precessing(m1, a1, tilt1, m2, a2, tilt2):
    q_inv = m1/m2
    A1 = 2. + (3.*q_inv/2.)
    A2 = 2. + 3./(2.*q_inv)
    S1_perp = a1*np.sin(tilt1)*m1*m1
    S2_perp = a2*np.sin(tilt2)*m2*m2
    Sp = np.maximum(A1*S2_perp, A2*S1_perp)
    chi_p = Sp/(A2*m1*m1)
    return chi_p

Generate KDE for $\chi_\mathrm{p}$ from the full run and the prior, for comparison 

In [4]:
# grid on which to evaluate kdes
chip_grid = np.linspace(0,1,500)

# kde for full run 
samples_full = td_samples['full']
m1_full, m2_full = m1m2_from_mtotq(samples_full['mtotal'], samples_full['q'])
chip_full_samples = chi_precessing(m1_full, samples_full['chi1'], samples_full['tilt1'],
                          m2_full, samples_full['chi2'], samples_full['tilt2'])

chip_full_kde_on_grid = gaussian_kde(chip_full_samples)(chip_grid) + \
                        gaussian_kde(-chip_full_samples)(chip_grid) + \
                        gaussian_kde(2-chip_full_samples)(chip_grid) 

# kde for prior 
samples_prior = td_samples['prior']
m1_prior, m2_prior = m1m2_from_mtotq(samples_prior['mtotal'], samples_prior['q'])
chip_prior_samples = chi_precessing(m1_prior, samples_prior['chi1'], samples_prior['tilt1'],
                          m2_prior, samples_prior['chi2'], samples_prior['tilt2'])

chip_prior_kde_on_grid = gaussian_kde(chip_prior_samples)(chip_grid) + \
                         gaussian_kde(-chip_prior_samples)(chip_grid) + \
                         gaussian_kde(2-chip_prior_samples)(chip_grid)

### Load in data: strain timeseries

Load in time-domain data from the detectors

In [5]:
root = '/home/simona.miller/gw190521-td/notebooks/simona_plots/'
data_dict =  np.load(root+'data_dict.npy', allow_pickle=True).item()
wh_data_dict =  np.load(root+'wh_data_dict.npy', allow_pickle=True).item()
time_dict_data =  np.load(root+'time_dict_for_data.npy', allow_pickle=True).item()

Load in max likelihood LVK reconstruction

In [6]:
LVC_maxL_strain_wh =  np.load(root+'LVC_maxL_strain_whitened.npy', allow_pickle=True).item()
LVC_maxL_strain =  np.load(root+'LVC_maxL_strain.npy', allow_pickle=True).item()
time_dict =  np.load(root+'time_dict.npy', allow_pickle=True).item()

Define cutoff times 

In [7]:
# 0M <-> seconds
t0_0M_dict = {}
t0_0M_geo = 1242442967.405764
dt_10M = 0.0127 # 10 M = 12.7 ms 
dt_1M = dt_10M/10.

tstart = 1242442966.9077148
tend = 1242442967.607715

ra = 6.07546535866838
dec = -0.8000357325337637

# define t_0M in each detector in seconds
for ifo in ['H1', 'L1', 'V1']: 
    
    t_delay = lal.TimeDelayFromEarthCenter(lal.cached_detector_by_prefix[ifo].location, ra, dec, t0_0M_geo)
    t0_0M_dict[ifo] = t0_0M_geo + t_delay
    
# Transform L1 into units of M
L1_LVC_maxL_strain = LVC_maxL_strain['L1']
L1_LVC_maxL_strain_wh = LVC_maxL_strain_wh['L1']
L1_maxL_times_M = (time_dict['L1'] - t0_0M_dict['L1'])/dt_1M

L1_LVC_strain = data_dict['L1']
L1_LVC_strain_wh = wh_data_dict['L1']
L1_times_M = (time_dict_data['L1'] - t0_0M_dict['L1'])/dt_1M

### Make figure

In [8]:
# Define times to plot
tc_to_plot = ['m40M', 'm20M', 'm12.5M', 'm10M', '15M', '20M', '30M']
n_to_plot = len(tc_to_plot)

# Cast strings to floats
tc_floats = []
for tc_str in tc_to_plot:
    if tc_str[0]=='m':
        tc_str_trimmed = tc_str[1:-1]
    else:
        tc_str_trimmed = tc_str[0:-1]     
    tc_float = -1*float(tc_str_trimmed) if tc_str[0]=='m' else float(tc_str_trimmed) 
    tc_floats.append(tc_float)

# Make figure 
fig, axes = plt.subplots(n_to_plot, 3, figsize=(15, 25*n_to_plot/8))

for j,tc in enumerate(tc_floats):
    
    # Lefthand plots: whitened waveforms with cutoff
    if j==0:
        top_ax = axes[j][0].twiny()
        top_ax.plot(L1_times_M*dt_1M, L1_LVC_strain_wh, color='k', alpha=0.2)
        top_ax.set_xlabel(r'$t-t_{0M, L1} [s] $', fontsize=15)
        top_ax.set_xlim(-55*dt_1M, 70*dt_1M)
        top_ax.grid(visible=False)
    else:
        axes[j][0].plot(L1_times_M, L1_LVC_strain_wh, color='k', alpha=0.2)
    axes[j][0].plot(L1_maxL_times_M[L1_maxL_times_M<tc], L1_LVC_maxL_strain_wh[L1_maxL_times_M<tc], color='C0')
    axes[j][0].plot(L1_maxL_times_M[L1_maxL_times_M>tc], L1_LVC_maxL_strain_wh[L1_maxL_times_M>tc], color='C1')
    axes[j][0].axvline(tc, ls='--', color='k')
    
    tc_str = tc_to_plot[j]
    lbl = tc_str.replace('m', '-') if tc_str[0]=='m' else tc_str
    axes[j][0].text(tc+1, 2.5, f'${lbl}$', color='k', fontsize=15)
    
    if j!=n_to_plot-1: 
        axes[j][0].set_xticklabels([])
    else: 
        axes[j][0].set_xlabel(r'$t-t_{0M, L1} [M]$', fontsize=15)
    axes[j][0].set_ylabel(r'$\sigma_\mathrm{noise}$ [L1]', fontsize=15)
    axes[j][0].grid(color='silver', ls=':', alpha=0.7)
    axes[j][0].set_xlim(-55, 70)
    axes[j][0].set_ylim(-3.5, 3.5)
        
    
    # Middle plots: colored waveforms with cutoff
    
    if j==0:
        top_ax = axes[j][1].twiny()
        top_ax.plot(L1_maxL_times_M[L1_maxL_times_M<tc]*dt_1M, L1_LVC_maxL_strain[L1_maxL_times_M<tc]/(1e-22), color='C0')
        top_ax.set_xlabel(r'$t-t_{0M, L1} [s] $', fontsize=15)
        top_ax.set_xlim(-200*dt_1M, 90*dt_1M)
        top_ax.grid(visible=False)
    else:
        axes[j][1].plot(L1_maxL_times_M[L1_maxL_times_M<tc], L1_LVC_maxL_strain[L1_maxL_times_M<tc]/(1e-22), color='C0')
    axes[j][1].plot(L1_maxL_times_M[L1_maxL_times_M>tc], L1_LVC_maxL_strain[L1_maxL_times_M>tc]/(1e-22), color='C1')
    axes[j][1].axvline(tc, ls='--', color='k')

    
    if j!=n_to_plot-1: 
        axes[j][1].set_xticklabels([])
    else: 
        axes[j][1].set_xlabel(r'$t-t_{0M, L1} [M]$', fontsize=15)
    axes[j][1].set_ylabel(r'$h/10^{-22}$ [L1]', fontsize=15)
    axes[j][1].grid(color='silver', ls=':', alpha=0.7)
    axes[j][1].set_xlim(-200, 90)
    axes[j][1].set_ylim(-5.2, 5.2)
    axes[j][1].text(tc+1, 4, f'${lbl}$', color='k', fontsize=15)
    
    axes[j][1].yaxis.set_label_position("right")
    axes[j][1].yaxis.tick_right()
    
    x0, y0, x1, y1 = axes[j][1].get_position().bounds
    axes[j][1].set_position([x0-0.0375, y0, x1, y1])
    
    # Lefthand plots: chi_p posteriors
    try:
        samples_insp = td_samples[f'insp {tc_str}']
        m1_insp, m2_insp = m1m2_from_mtotq(samples_insp['mtotal'], samples_insp['q'])
        chip_insp = chi_precessing(m1_insp, samples_insp['chi1'], samples_insp['tilt1'], 
                                   m2_insp, samples_insp['chi2'], samples_insp['tilt2'])
    except: 
        samples_insp = td_samples['prior']
        mask = np.random.choice(len(samples_insp), size=len(td_samples[f'rd {tc_str}']))
        m1_insp, m2_insp = m1m2_from_mtotq(samples_insp['mtotal'][mask], samples_insp['q'][mask])
        chip_insp = chi_precessing(m1_insp, samples_insp['chi1'][mask], samples_insp['tilt1'][mask], 
                                   m2_insp, samples_insp['chi2'][mask], samples_insp['tilt2'][mask])
    
    samples_rd = td_samples[f'rd {tc_str}']
    m1_rd, m2_rd = m1m2_from_mtotq(samples_rd['mtotal'], samples_rd['q'])
    chip_rd = chi_precessing(m1_rd, samples_rd['chi1'], samples_rd['tilt1'],
                                m2_rd, samples_rd['chi2'], samples_rd['tilt2'])
    
    axes[j][2].hist(chip_insp, histtype='step', bins=np.linspace(0,1,30), lw=1.5, 
                    color='C0', label='before cutoff', density=True)
    axes[j][2].hist(chip_rd, histtype='step', bins=np.linspace(0,1,30), lw=1.5, color='C1', 
                    label='after cutoff', density=True)
    
    axes[j][2].plot(chip_grid, chip_full_kde_on_grid, color='k', label='full waveform')
    axes[j][2].plot(chip_grid, chip_prior_kde_on_grid, color='k', ls=':', lw=2, label='prior')
    
    axes[j][2].yaxis.set_label_position("right")
    axes[j][2].set_ylabel(r'$p(\chi_\mathrm{p})$', fontsize=15)
    axes[j][2].yaxis.tick_right()
    if j!=n_to_plot-1: 
        axes[j][2].set_xticklabels([])
    else: 
        axes[j][2].set_xlabel(r'$\chi_\mathrm{p}$', fontsize=15)
    axes[j][2].set_xlim(0,1)
    axes[j][2].set_ylim(0,2.3)
    axes[j][2].grid(color='silver', ls=':', alpha=0.7)
    
for ax in axes: 
    for a in ax:
        x0, y0, x1, y1 = a.get_position().bounds
        a.set_position([x0, y0, x1, y1+0.03/n_to_plot])
    
axes[0][2].legend(fontsize=13, bbox_to_anchor=(1.2, 1.05))

plt.savefig('figure_01.pdf', bbox_inches='tight')
plt.show()

RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full report generated by latex:
This is pdfTeX, Version 3.1415926-2.5-1.40.14 (TeX Live 2013)
 restricted \write18 enabled.
entering extended mode
(../41e3affc34550e765935caf5a875fad6.tex
LaTeX2e <2011/06/27>
Babel <v3.8m> and hyphenation patterns for english, dumylang, nohyphenation, lo
aded.
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2007/10/19 v1.4h Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))

! LaTeX Error: File `type1cm.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.6 \usepackage
               {type1ec}^^M
No pages of output.
Transcript written on 41e3affc34550e765935caf5a875fad6.log.




Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f3344d12940> (for post_execute):


RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full report generated by latex:
This is pdfTeX, Version 3.1415926-2.5-1.40.14 (TeX Live 2013)
 restricted \write18 enabled.
entering extended mode
(../41e3affc34550e765935caf5a875fad6.tex
LaTeX2e <2011/06/27>
Babel <v3.8m> and hyphenation patterns for english, dumylang, nohyphenation, lo
aded.
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2007/10/19 v1.4h Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))

! LaTeX Error: File `type1cm.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.6 \usepackage
               {type1ec}^^M
No pages of output.
Transcript written on 41e3affc34550e765935caf5a875fad6.log.




RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full report generated by latex:
This is pdfTeX, Version 3.1415926-2.5-1.40.14 (TeX Live 2013)
 restricted \write18 enabled.
entering extended mode
(../41e3affc34550e765935caf5a875fad6.tex
LaTeX2e <2011/06/27>
Babel <v3.8m> and hyphenation patterns for english, dumylang, nohyphenation, lo
aded.
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2007/10/19 v1.4h Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))

! LaTeX Error: File `type1cm.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.6 \usepackage
               {type1ec}^^M
No pages of output.
Transcript written on 41e3affc34550e765935caf5a875fad6.log.




<Figure size 1500x2187.5 with 23 Axes>

### Make gif version of figure including all timeslices

In [ ]:
# Define times to plot (here, all times)
tc_to_plot = tcutoffs

# Cast strings to floats
tc_floats = []
for tc_str in tc_to_plot:
    if tc_str[0]=='m':
        tc_str_trimmed = tc_str[1:-1]
    else:
        tc_str_trimmed = tc_str[0:-1]     
    tc_float = -1*float(tc_str_trimmed) if tc_str[0]=='m' else float(tc_str_trimmed) 
    tc_floats.append(tc_float)

# Make frames of gif
for j,tc in enumerate(tc_floats):
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 25/8))
    
    # Lefthand plots: whitened waveforms with cutoff
    top_ax = axes[0].twiny()
    top_ax.plot(L1_times_M*dt_1M, L1_LVC_strain_wh, color='k', alpha=0.2)
    top_ax.set_xlabel(r'$t-t_{0M, L1} [s] $', fontsize=15)
    top_ax.set_xlim(-55*dt_1M, 70*dt_1M)
    top_ax.grid(visible=False)
    
    axes[0].plot(L1_maxL_times_M[L1_maxL_times_M<tc], L1_LVC_maxL_strain_wh[L1_maxL_times_M<tc], color='C0')
    axes[0].plot(L1_maxL_times_M[L1_maxL_times_M>tc], L1_LVC_maxL_strain_wh[L1_maxL_times_M>tc], color='C1')
    axes[0].axvline(tc, ls='--', color='k')
    
    tc_str = tc_to_plot[j]
    lbl = tc_str.replace('m', '-') if tc_str[0]=='m' else tc_str
    axes[0].text(tc+1, 2.5, f'${lbl}$', color='k', fontsize=15)
    
    axes[0].set_xlabel(r'$t-t_{0M, L1} [M]$', fontsize=15)
    axes[0].set_ylabel(r'$\sigma_\mathrm{noise}$ [L1]', fontsize=15)
    axes[0].grid(color='silver', ls=':', alpha=0.7)
    axes[0].set_xlim(-55, 70)
    axes[0].set_ylim(-3.5, 3.5)
        
    
    # Middle plots: colored waveforms with cutoff
    
    top_ax2 = axes[1].twiny()
    top_ax2.plot(L1_maxL_times_M[L1_maxL_times_M<tc]*dt_1M, L1_LVC_maxL_strain[L1_maxL_times_M<tc]/(1e-22), color='C0')
    top_ax2.set_xlabel(r'$t-t_{0M, L1} [s] $', fontsize=15)
    top_ax2.set_xlim(-200*dt_1M, 90*dt_1M)
    top_ax2.grid(visible=False)
    
    axes[1].plot(L1_maxL_times_M[L1_maxL_times_M>tc], L1_LVC_maxL_strain[L1_maxL_times_M>tc]/(1e-22), color='C1')
    axes[1].axvline(tc, ls='--', color='k')
    
    axes[1].set_xlabel(r'$t-t_{0M, L1} [M]$', fontsize=15)
    axes[1].set_ylabel(r'$h/10^{-22}$ [L1]', fontsize=15)
    axes[1].grid(color='silver', ls=':', alpha=0.7)
    axes[1].set_xlim(-200, 90)
    axes[1].set_ylim(-5.2, 5.2)
    axes[1].text(tc+1, 4, f'${lbl}$', color='k', fontsize=15)
    
    axes[1].yaxis.set_label_position("right")
    axes[1].yaxis.tick_right()
    
    x0, y0, x1, y1 = axes[1].get_position().bounds
    axes[1].set_position([x0-0.0375, y0, x1, y1])
    
    # Lefthand plots: chi_p posteriors
    try:
        samples_insp = td_samples[f'insp {tc_str}']
        m1_insp, m2_insp = m1m2_from_mtotq(samples_insp['mtotal'], samples_insp['q'])
        chip_insp = chi_precessing(m1_insp, samples_insp['chi1'], samples_insp['tilt1'], 
                                   m2_insp, samples_insp['chi2'], samples_insp['tilt2'])
    except: 
        samples_insp = td_samples['prior']
        mask = np.random.choice(len(samples_insp), size=len(td_samples[f'rd {tc_str}']))
        m1_insp, m2_insp = m1m2_from_mtotq(samples_insp['mtotal'][mask], samples_insp['q'][mask])
        chip_insp = chi_precessing(m1_insp, samples_insp['chi1'][mask], samples_insp['tilt1'][mask], 
                                   m2_insp, samples_insp['chi2'][mask], samples_insp['tilt2'][mask])
    
    samples_rd = td_samples[f'rd {tc_str}']
    m1_rd, m2_rd = m1m2_from_mtotq(samples_rd['mtotal'], samples_rd['q'])
    chip_rd = chi_precessing(m1_rd, samples_rd['chi1'], samples_rd['tilt1'],
                                m2_rd, samples_rd['chi2'], samples_rd['tilt2'])
    
    axes[2].hist(chip_insp, histtype='step', bins=np.linspace(0,1,30), lw=1.5, 
                    color='C0', label='before cutoff', density=True)
    axes[2].hist(chip_rd, histtype='step', bins=np.linspace(0,1,30), lw=1.5, color='C1', 
                    label='after cutoff', density=True)
    
    axes[2].plot(chip_grid, chip_full_kde_on_grid, color='k', label='full waveform')
    axes[2].plot(chip_grid, chip_prior_kde_on_grid, color='k', ls=':', lw=2, label='prior')
    
    axes[2].yaxis.set_label_position("right")
    axes[2].set_ylabel(r'$p(\chi_\mathrm{p})$', fontsize=15)
    axes[2].yaxis.tick_right()
    axes[2].set_xlabel(r'$\chi_\mathrm{p}$', fontsize=15)
    axes[2].set_xlim(0,1)
    axes[2].set_ylim(0,2.3)
    axes[2].grid(color='silver', ls=':', alpha=0.7)
    
    axes[2].legend(fontsize=13, bbox_to_anchor=(1.2, 1.05))

    plt.savefig(f'for_figure_01_gif/frame_{tc_str}.png', dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
# Assemble into a gif
frames = []
for tc in tcutoffs:
    image = imageio.v2.imread(f'for_figure_01_gif/frame_{tc}.png')
    frames.append(image)
fps = 1.5
imageio.mimsave('figure_01.gif', frames, duration = (1000 * 1.0/fps))  